In [1]:
# coding: utf-8

from  __future__ import absolute_import
from __future__ import print_function
from ImageDataGeneratorCustom import ImageDataGeneratorCustom
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import tensorflow_hub as hub
import shutil
import random
from PIL import Image
import pickle
import shutil

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


print(tf.__version__)

model_path = "./deep_ranking"

2.8.0


In [2]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224,3))
feature_extractor.trainable = False
model1 = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(2048, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    #L2 normalization
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])


model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(48, kernel_size=(8, 8),strides=(16,16), padding='same',input_shape=(224, 224,3)),
    tf.keras.layers.MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

model3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(48, kernel_size=(8, 8),strides=(32,32), padding='same',input_shape=(224, 224,3)),
    tf.keras.layers.MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

merge_one = concatenate([model2.output, model3.output])

merge_two = concatenate([merge_one, model1.output])

emb = Dense(1024)(merge_two)
l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

deep_rank_model = Model(inputs=[model2.input, model3.input, model1.input], outputs=l2_norm_final)

deep_rank_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d_input (InputLayer)      [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1_input (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 48)   9264        ['conv2d_input[0][0]']           
                                                                                              

In [3]:
batch_size = 96
# batch_size = 8
# batch_size *= 3


In [4]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
    loss =  tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    for i in range(0,batch_size,3):
        try:
            q_embedding = y_pred[i+0]
            p_embedding =  y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p =  K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = (loss + g + D_q_p - D_q_n )            
        except:
            continue
    loss = loss/(batch_size/3)
    zero = tf.constant(0.0, shape=[1], dtype=tf.float32)
    return tf.maximum(loss,zero)



In [5]:
pred = deep_rank_model.predict((np.random.rand(24,224,224,3),np.random.rand(24,224,224,3),np.random.rand(24,224,224,3)))
print(pred.shape)
_loss_tensor(None,pred).numpy()
pred

(24, 1024)


array([[-0.02979908, -0.03597136,  0.0584613 , ...,  0.04125079,
         0.01803314,  0.02594753],
       [-0.03840109, -0.03160903,  0.05542181, ...,  0.03299639,
         0.01438763,  0.0278038 ],
       [-0.02918385, -0.03983974,  0.06002127, ...,  0.03711349,
         0.00500056,  0.02223518],
       ...,
       [-0.00929163, -0.01800902,  0.05513421, ...,  0.03550531,
         0.01317814,  0.01035433],
       [-0.02102469, -0.03468908,  0.05799765, ...,  0.04269026,
         0.01595426,  0.01185478],
       [-0.03755579, -0.03214884,  0.05447798, ...,  0.03068008,
         0.01553553,  0.02549384]], dtype=float32)

In [6]:

gen1 = ImageDataGenerator(rescale=1./255)
gen1 = gen1.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen2 = ImageDataGenerator(rescale=1./255)
gen2 = gen2.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)
gen3 = ImageDataGenerator(rescale=1./255)
gen3 = gen3.flow_from_directory(batch_size=batch_size,directory="./triplets/",shuffle=False,target_size=(224,224),class_mode=None)

class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1, input_gen2, target_gen):
        self.gen1 = input_gen1
        self.gen2 = input_gen2
        self.gen3 = target_gen

        assert len(input_gen1) == len(input_gen2) == len(target_gen)

    def __len__(self):
        return len(self.gen1)

    def __getitem__(self, i):
        x1 = self.gen1[i]
        x2 = self.gen2[i]
        y = self.gen3[i]

        return [x1, x2, y], y

    def on_epoch_end(self):
        self.gen1.on_epoch_end()
        self.gen2.on_epoch_end()
        self.gen3.on_epoch_end()

my_gen = JoinedGen(gen1, gen2, gen3)


Found 178560 images belonging to 1 classes.
Found 178560 images belonging to 1 classes.
Found 178560 images belonging to 1 classes.


In [7]:
deep_rank_model.compile(loss=_loss_tensor, optimizer='adam')

#copy the last train triplet a few times first, so that there are 59520 of them
train_steps_per_epoch = int((178560)/batch_size)
train_epochs = 2
deep_rank_model.fit_generator(my_gen,
                        steps_per_epoch=train_steps_per_epoch,
                        epochs=train_epochs
                        )




C:\Users\Timothe\AppData\Local\Temp/ipykernel_23540/3025222012.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  deep_rank_model.fit_generator(my_gen,


Epoch 1/2
 226/1860 [==>...........................] - ETA: 11:09 - loss: nan

KeyboardInterrupt: 

In [ ]:

model_path = "deepranking"
tf.saved_model.save(deep_rank_model, model_path)
